In [1]:

import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [5]:
df=pd.read_excel("combined.xlsx")

df.head()

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


In [6]:
df.shape

(47840, 5)

In [9]:
train, test = train_test_split(df, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.25, random_state=42)


scaler_x = StandardScaler()
scaler_y = StandardScaler()

train_x = scaler_x.fit_transform(train.iloc[:,:-1].values)
train_y = scaler_y.fit_transform(train.iloc[:,-1].values.reshape(-1,1))

val_x = scaler_x.transform(val.iloc[:,:-1].values)
val_y = scaler_y.transform(val.iloc[:,-1].values.reshape(-1,1))

test_x = scaler_x.transform(test.iloc[:,:-1].values)
test_y = scaler_y.transform(test.iloc[:,-1].values.reshape(-1,1))

In [10]:
train_x


array([[ 1.28458868,  1.56703203, -0.26703263, -0.05892276],
       [-0.81269777, -0.99184572,  1.41444384, -0.71268882],
       [ 1.36087797,  1.35110955, -0.53943182, -0.99845449],
       ...,
       [ 1.96985393,  1.85126454, -0.57474282, -1.05327764],
       [ 0.07600561,  0.26600091,  0.5098095 ,  0.31044821],
       [ 0.09072811,  0.24558642,  0.18696602,  0.66679869]])

In [11]:
train_y

array([[-1.6221037 ],
       [ 1.02173861],
       [-1.28738836],
       ...,
       [-1.36858458],
       [-0.09573164],
       [-0.11792917]])

In [24]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(4,36)
        self.fc2 = nn.Linear(36,69)
        self.fc3 = nn.Linear(69,10)
        self.fc4 = nn.Linear(10,1)
    def forward(self,x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x=self.fc4(x)
        return x


In [25]:
net = Net()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

In [26]:
for epoch in range(690):
    optimizer.zero_grad()
    outputs = net(torch.Tensor(train_x))
    loss = criterion(outputs, torch.Tensor(train_y))
    loss.backward()
    optimizer.step()

    if epoch % 69 == 0:
        val_outputs = net(torch.Tensor(val_x))
        val_loss = criterion(val_outputs, torch.Tensor(val_y))
        print("Epoch: {}, Loss: {}, Validation Loss: {}".format(epoch, loss.item(), val_loss.item()))

Epoch: 0, Loss: 1.1432256698608398, Validation Loss: 0.9493564367294312
Epoch: 69, Loss: 0.060140594840049744, Validation Loss: 0.059607017785310745
Epoch: 138, Loss: 0.057907767593860626, Validation Loss: 0.05746256560087204
Epoch: 207, Loss: 0.0566716231405735, Validation Loss: 0.056320350617170334
Epoch: 276, Loss: 0.055623091757297516, Validation Loss: 0.05529699847102165
Epoch: 345, Loss: 0.05449569225311279, Validation Loss: 0.05430310219526291
Epoch: 414, Loss: 0.05308057367801666, Validation Loss: 0.052947767078876495
Epoch: 483, Loss: 0.05128920078277588, Validation Loss: 0.051235537976026535
Epoch: 552, Loss: 0.050168175250291824, Validation Loss: 0.05028075352311134
Epoch: 621, Loss: 0.04874332621693611, Validation Loss: 0.04864150285720825


In [27]:
test_outputs = net(torch.Tensor(test_x))
print(test_outputs)

tensor([[-1.1920],
        [ 0.7842],
        [-0.8981],
        ...,
        [-1.3967],
        [ 0.3131],
        [ 1.5847]], grad_fn=<AddmmBackward0>)


In [28]:
test_loss = criterion(test_outputs,torch.Tensor(test_y))
print(f"Test loss: {test_loss.item()}")

Test loss: 0.04818805679678917
